# Week 2 Programming Assignment 

Remark: 

Please upload your solutions of this assignment to Canvas with a file named "Programming_Assignment_2 _yourname.ipynb" before 11:59pm May 30.

### **Problem 1 (4 pt).** Use stochastic gradient descent method to train MNIST with the logisitc regression model to achieve at least 92% test accuracy. Print the results with the following format:

   "Epoch: i, Training accuracy: $a_i$, Test accuracy: $b_i$"

where $i=1,2,3,...$ means the $i$-th epoch,  $a_i$ and $b_i$ are the training accuracy and test accuracy computed at the end of $i$-th epoch.

In [3]:
# Code for achieving 92 % test accuracy with Stochastic Gradient Descent Method to train MNIST

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

def model(input_size,num_classes):
    return nn.Linear(input_size,num_classes)

input_size = 784
num_classes = 10

my_model = model(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(my_model.parameters(), lr=0.1)

MNIST_transform = torchvision.transforms.ToTensor()

trainset = torchvision.datasets.MNIST(root='./data', train= True, download=True, transform=MNIST_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train= False, download=True, transform=MNIST_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False) 

# Works for num_epoch = 7 also, but I decided to go with 10
# to test what happens with train accuracy
num_epochs = 10

for epoch in range(num_epochs):
    count_test, count_train = 0, 0
    correct_test, correct_train = 0, 0
    Accuracy_test, Accuracy_train = 0, 0
    for i, (images, labels) in enumerate(trainloader):
        images = images.reshape(images.size(0), 28*28)
        outputs = my_model(images)
        loss = criterion(outputs, labels) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    for i, (images, labels) in enumerate(trainloader):
        images = images.reshape(images.size(0), 1*28*28)
        outputs = my_model(images)
        p_max, predicted = torch.max(outputs, 1) 
        count_train += labels.size(0)
        correct_train += (predicted == labels).sum()
        Accuracy_train = 100 * float(correct_train) / count_train
        
    for i, (images, labels) in enumerate(testloader):
        images = images.reshape(images.size(0), 1*28*28)
        outputs = my_model(images)
        p_max, predicted = torch.max(outputs, 1) 
        count_test += labels.size(0)
        correct_test += (predicted == labels).sum()
        Accuracy_test = 100 * float(correct_test) / count_test
        
    print('Epoch: {}, Test accuracy: {}, Train accuracy: {}'.format(epoch+1, Accuracy_test, Accuracy_train))

Epoch: 1, Test accuracy: 90.24, Train accuracy: 89.66166666666666
Epoch: 2, Test accuracy: 91.13, Train accuracy: 90.54333333333334
Epoch: 3, Test accuracy: 91.4, Train accuracy: 91.11
Epoch: 4, Test accuracy: 91.94, Train accuracy: 91.505
Epoch: 5, Test accuracy: 91.86, Train accuracy: 91.61666666666666
Epoch: 6, Test accuracy: 91.89, Train accuracy: 91.74166666666666
Epoch: 7, Test accuracy: 92.12, Train accuracy: 91.9
Epoch: 8, Test accuracy: 91.98, Train accuracy: 92.02166666666666
Epoch: 9, Test accuracy: 92.12, Train accuracy: 92.08
Epoch: 10, Test accuracy: 92.1, Train accuracy: 92.12833333333333


### **Problem 2 (6 pts).** Extract the subset of data which are labeled with 0,1,3,4,7 from MNIST dataset. Use both full gradient descent method and stochastic gradient descent method to train this subset with the logisitc regression model to achieve the training and test accuracy as high as possible. Print the results with the following format:

* For full gradient descent method, print:

    "Full gradient descent, Epoch: i, Training accuracy: $a_i$, Test accuracy: $b_i$"


* For stochastic gradient descent method, print:

    "Stochastic gradient descent, Epoch: i, Training accuracy: $a_i$, Test accuracy: $b_i$"

where $i=1,2,3,...$ means the $i$-th epoch,  $a_i$ and $b_i$ are the training accuracy and test accuracy computed at the end of $i$-th epoch.

In [1]:
# The next two segments of the code is for a specific subset of MNIST dataset

# Code for achieving 98 % test accuracy with Stochastic Gradient Descent Method

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

def get_indices(dataset):
    indices =  []
    for i in range(len(dataset.targets)):
        if dataset.targets[i] == 0:
            dataset.targets[i] = 0
            indices.append(i)
        elif dataset.targets[i] == 1:
            dataset.targets[i] = 1
            indices.append(i)
        elif dataset.targets[i] == 3:
            dataset.targets[i] = 2
            indices.append(i)
        elif dataset.targets[i] == 4:
            dataset.targets[i] = 3
            indices.append(i)
        elif dataset.targets[i] == 7:
            dataset.targets[i] = 4
            indices.append(i)
    return indices

def model(input_size,num_classes):
    return nn.Linear(input_size,num_classes)

input_size = 784
num_classes = 5

my_model = model(input_size,num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(my_model.parameters(), lr=0.25)

MNIST_transform = torchvision.transforms.ToTensor()

trainset = torchvision.datasets.MNIST(root='./data', train= True, download=True, transform=MNIST_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=200, sampler=torch.utils.data.sampler.SubsetRandomSampler(get_indices(trainset)))

testset = torchvision.datasets.MNIST(root='./data', train= False, download=True, transform=MNIST_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, sampler=torch.utils.data.sampler.SubsetRandomSampler(get_indices(testset)))

num_epochs = 20

for epoch in range(num_epochs):
    correct_train, correct_test = 0, 0
    count_train, count_test = 0, 0
    Accuracy_test, Accuracy_train = 0, 0
    for i, (images, labels) in enumerate(trainloader):
        images = images.reshape(images.size(0), 28*28)
        outputs = my_model(images)
        loss = criterion(outputs, labels) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    for i, (images, labels) in enumerate(trainloader):
        images = images.reshape(images.size(0), 1*28*28)
        outputs = my_model(images)
        p_max, predicted = torch.max(outputs, 1) 
        count_train += labels.size(0)
        correct_train += (predicted == labels).sum()
        Accuracy_train = 100 * float(correct_train) / count_train
        
    for i, (images, labels) in enumerate(testloader):
        images = images.reshape(images.size(0), 1*28*28)
        outputs = my_model(images)
        p_max, predicted = torch.max(outputs, 1) 
        count_test += labels.size(0)
        correct_test += (predicted == labels).sum()
        Accuracy_test = 100 * float(correct_test) / count_test

    print('Stochastic Gradient Descent Method, Epoch: {}, Test accuracy: {}, Train accuracy: {}'.format(epoch+1, Accuracy_test, Accuracy_train))

Stochastic Gradient Descent Method, Epoch: 1, Test accuracy: 97.56572541382668, Train accuracy: 97.00999902922047
Stochastic Gradient Descent Method, Epoch: 2, Test accuracy: 97.8188899707887, Train accuracy: 97.42096236611333
Stochastic Gradient Descent Method, Epoch: 3, Test accuracy: 97.95520934761441, Train accuracy: 97.6151182733068
Stochastic Gradient Descent Method, Epoch: 4, Test accuracy: 98.09152872444011, Train accuracy: 97.74779147655568
Stochastic Gradient Descent Method, Epoch: 5, Test accuracy: 98.18889970788705, Train accuracy: 97.877228748018
Stochastic Gradient Descent Method, Epoch: 6, Test accuracy: 98.18889970788705, Train accuracy: 97.87399281623144
Stochastic Gradient Descent Method, Epoch: 7, Test accuracy: 98.1110029211295, Train accuracy: 97.96783483804161
Stochastic Gradient Descent Method, Epoch: 8, Test accuracy: 98.16942551119766, Train accuracy: 98.03902533734589
Stochastic Gradient Descent Method, Epoch: 9, Test accuracy: 98.24732229795521, Train accurac

In [1]:
# Code for achieving 96 % test accuracy with Full Gradient Descent Method

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

def get_indices(dataset):
    indices =  []
    for i in range(len(dataset.targets)):
        if dataset.targets[i] == 0:
            dataset.targets[i] = 0
            indices.append(i)
        elif dataset.targets[i] == 1:
            dataset.targets[i] = 1
            indices.append(i)
        elif dataset.targets[i] == 3:
            dataset.targets[i] = 2
            indices.append(i)
        elif dataset.targets[i] == 4:
            dataset.targets[i] = 3
            indices.append(i)
        elif dataset.targets[i] == 7:
            dataset.targets[i] = 4
            indices.append(i)
    return indices

def model(input_size,num_classes):
    return nn.Linear(input_size,num_classes)

input_size = 784
num_classes = 5

my_model = model(input_size,num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(my_model.parameters(), lr=0.25)

MNIST_transform = torchvision.transforms.ToTensor()

trainset = torchvision.datasets.MNIST(root='./data', train= True, download=True, transform=MNIST_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=60000, sampler=torch.utils.data.sampler.SubsetRandomSampler(get_indices(trainset)))

testset = torchvision.datasets.MNIST(root='./data', train= False, download=True, transform=MNIST_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, sampler=torch.utils.data.sampler.SubsetRandomSampler(get_indices(testset)))

num_epochs = 20

for epoch in range(num_epochs):
    correct_train, correct_test = 0, 0
    count_train, count_test = 0, 0
    Accuracy_test, Accuracy_train = 0, 0
    for i, (images, labels) in enumerate(trainloader):
        images = images.reshape(images.size(0), 28*28)
        outputs = my_model(images)
        loss = criterion(outputs, labels) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    for i, (images, labels) in enumerate(trainloader):
        images = images.reshape(images.size(0), 1*28*28)
        outputs = my_model(images)
        p_max, predicted = torch.max(outputs, 1) 
        count_train += labels.size(0)
        correct_train += (predicted == labels).sum()
        Accuracy_train = 100 * float(correct_train) / count_train
        
    for i, (images, labels) in enumerate(testloader):
        images = images.reshape(images.size(0), 1*28*28)
        outputs = my_model(images)
        p_max, predicted = torch.max(outputs, 1) 
        count_test += labels.size(0)
        correct_test += (predicted == labels).sum()
        Accuracy_test = 100 * float(correct_test) / count_test

    print('Full Gradient Descent Method, Epoch: {}, Test accuracy: {}, Train accuracy: {}'.format(epoch+1, Accuracy_test, Accuracy_train))

Full Gradient Descent Method, Epoch: 1, Test accuracy: 83.0379746835443, Train accuracy: 81.19923632009838
Full Gradient Descent Method, Epoch: 2, Test accuracy: 93.16455696202532, Train accuracy: 91.88104714752613
Full Gradient Descent Method, Epoch: 3, Test accuracy: 93.94352482960078, Train accuracy: 92.87124227421286
Full Gradient Descent Method, Epoch: 4, Test accuracy: 94.76144109055501, Train accuracy: 93.6996408115717
Full Gradient Descent Method, Epoch: 5, Test accuracy: 95.01460564751704, Train accuracy: 93.99087467236191
Full Gradient Descent Method, Epoch: 6, Test accuracy: 95.24829600778968, Train accuracy: 94.17855871598226
Full Gradient Descent Method, Epoch: 7, Test accuracy: 95.42356377799416, Train accuracy: 94.3144678510177
Full Gradient Descent Method, Epoch: 8, Test accuracy: 95.55988315481986, Train accuracy: 94.4374332589069
Full Gradient Descent Method, Epoch: 9, Test accuracy: 95.69620253164557, Train accuracy: 94.56687053036921
Full Gradient Descent Method, Ep